# Exercise: Implementing Matrix Factorization from Scratch

**Course:** Recommender Systems <br>
**Professor:** Guilherme MEDEIROS MACHADO <br>
**Topic:** Collaborative Filtering with Matrix Factorization

---

## Goal of the Exercise

The objective of this exercise is to build a movie recommender system by implementing the **Matrix Factorization** algorithm from scratch using Python. We will use the famous **MovieLens 100k** dataset. By the end of this notebook, you will have:

1.  Understood the theoretical foundations of matrix factorization.
2.  Implemented the algorithm using **Stochastic Gradient Descent (SGD)**.
3.  Trained your model on real-world movie rating data.
4.  Evaluated your model's performance using Root Mean Squared Error (RMSE).
5.  Generated personalized top-10 movie recommendations for a specific user.

This exercise forbids the use of pre-built matrix factorization libraries (like `surprise`, `lightfm`, etc.) to ensure you gain a deep understanding of the inner workings of the algorithm.

---

## The Dataset: MovieLens 100k

We will be using the MovieLens 100k dataset, a classic dataset in the recommender systems community. It contains:
* 100,000 ratings (1-5) from...
* 943 users on...
* 1682 movies.

You will need two files from this dataset:
* `u.data`: The full dataset of 100k ratings. Each row is in the format: `user_id`, `item_id`, `rating`, `timestamp`.
* `u.item`: Information about the movies (items). Each row contains the `item_id`, `movie_title`, and other metadata. We'll use it to get the movie names for our final recommendations.

Let's start by downloading and exploring the data.

In [1]:
import pandas as pd
import numpy as np
import os
from urllib.request import urlretrieve
import zipfile

# --- Download the dataset if it doesn't exist ---
if not os.path.exists('ml-100k'):
    print("Downloading MovieLens 100k dataset...")
    url = 'http://files.grouplens.org/datasets/movielens/ml-100k.zip'
    urlretrieve(url, 'ml-100k.zip')
    with zipfile.ZipFile('ml-100k.zip', 'r') as zip_ref:
        zip_ref.extractall()
    print("Download and extraction complete.")

# --- Load the data ---
# u.data contains the ratings
data_cols = ['user_id', 'item_id', 'rating', 'timestamp']
ratings_df = pd.read_csv('ml-100k/u.data', sep='\t', names=data_cols)

# u.item contains movie titles
item_cols = ['item_id', 'title'] + [f'col{i}' for i in range(22)] # Remaining columns are not needed
movies_df = pd.read_csv('ml-100k/u.item', sep='|', names=item_cols, encoding='latin-1', usecols=['item_id', 'title'])

# Merge the two dataframes to have movie titles and ratings in one place
df = pd.merge(ratings_df, movies_df, on='item_id')

print("Data loaded successfully!")
df.head()

Data loaded successfully!


,user_id,item_id,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,186,302,3,891717742,L.A. Confidential (1997)
2,22,377,1,878887116,Heavyweights (1994)
3,244,51,2,880606923,Legends of the Fall (1994)
4,166,346,1,886397596,Jackie Brown (1997)


---

## Part 1: Data Preparation

The raw data is a list of ratings. For matrix factorization, it's conceptually easier to think of our data as a large **user-item interaction matrix**, let's call it $R$. In this matrix:
* The rows represent users.
* The columns represent movies (items).
* The value at cell $(u, i)$, denoted $R_{ui}$, is the rating user $u$ gave to movie $i$.

This matrix is typically very **sparse**, as most users have only rated a small fraction of the available movies.

Let's create this matrix using a Pandas pivot table. This will also help us determine the number of unique users and movies.

In [2]:
# TODO:Your code here
from sklearn.model_selection import train_test_split

    
def create_user_item_matrix(df):
    """
    Creates the user-item interaction matrix from the dataframe.

    Args:
        df (pd.DataFrame): The dataframe containing user_id, item_id, and rating.

    Returns:
        pd.DataFrame: A user-item matrix with users as rows, items as columns, and ratings as values.
                       NaNs indicate that a user has not rated an item.
    """

    R_df = df.pivot_table(
    index="user_id",
    columns="item_id",
    values="rating",
    aggfunc="mean")
    
    R_np = R_df.to_numpy().astype(float)
    
    rng = np.random.default_rng(42)
    known_mask = ~np.isnan(R_np)
    u_idx, i_idx = np.where(known_mask)
    perm = rng.permutation(len(u_idx))
    
    test_size = int(0.20 * len(u_idx))
    test_sel = perm[:test_size]
    train_sel = perm[test_size:]
    
    R_train = np.full_like(R_np, np.nan)
    R_test  = np.full_like(R_np, np.nan)
    
    R_train[u_idx[train_sel], i_idx[train_sel]] = R_np[u_idx[train_sel], i_idx[train_sel]]
    R_test[u_idx[test_sel],   i_idx[test_sel]]   = R_np[u_idx[test_sel],   i_idx[test_sel]]
    
    return (R_train,R_test)
    # TODO: Create a pivot table.
    # The index should be 'user_id', columns 'item_id', and values 'rating'.

---

## Part 2: The Theory of Matrix Factorization

The core idea is to **decompose** our large, sparse user-item matrix $R$ (size $m \times n$) into two smaller, dense matrices:
1.  A **user-feature matrix** $P$ (size $m \times k$).
2.  An **item-feature matrix** $Q$ (size $n \times k$).

Here, $k$ is the number of **latent factors**, which is a hyperparameter we choose. These latent factors represent hidden characteristics of users and items. For movies, a factor might represent the "amount of comedy" vs. "drama", or "blockbuster" vs. "indie film". For users, a factor might represent their preference for these characteristics.



The prediction of a rating $\hat{r}_{ui}$ that user $u$ would give to item $i$ is calculated by the dot product of the user's latent vector $p_u$ and the item's latent vector $q_i$:

$$\hat{r}_{ui} = p_u \cdot q_i^T = \sum_{k=1}^{K} p_{uk} q_{ik}$$

Our goal is to find the matrices $P$ and $Q$ such that their product $P \cdot Q^T$ is as close as possible to the known ratings in our original matrix $R$. We formalize this using a **loss function**. A common choice is the sum of squared errors, with **regularization** to prevent overfitting:

$$L = \sum_{(u,i) \in \mathcal{K}} (r_{ui} - \hat{r}_{ui})^2 + \lambda \left( \sum_{u} ||p_u||^2 + \sum_{i} ||q_i||^2 \right)$$

Where:
* $\mathcal{K}$ is the set of $(u, i)$ pairs for which the rating $r_{ui}$ is known.
* $\lambda$ is the regularization parameter, another hyperparameter.

---

## Part 3: The Algorithm - Stochastic Gradient Descent (SGD)

To minimize our loss function $L$, we will use **Stochastic Gradient Descent (SGD)**. Instead of calculating the gradient over all known ratings (which is computationally expensive), SGD iterates through each known rating one by one and updates the parameters in the direction that minimizes the error for that single rating.

For each known rating $r_{ui}$:
1.  Calculate the prediction error: $e_{ui} = r_{ui} - \hat{r}_{ui}$
2.  Update the user and item latent vectors ($p_u$ and $q_i$) using the following update rules:

$$p_u \leftarrow p_u + \alpha \cdot (e_{ui} \cdot q_i - \lambda \cdot p_u)$$
$$q_i \leftarrow q_i + \alpha \cdot (e_{ui} \cdot p_u - \lambda \cdot q_i)$$

Where:
* $\alpha$ is the **learning rate**, a hyperparameter that controls the step size.

We repeat this process for a fixed number of **epochs** (iterations over the entire training dataset).

---

## Part 4: Step-by-Step Implementation

Let's build our model. First, we need to split our data into a training and a testing set.

### 4.1 Initialization

We need to initialize our user-feature matrix $P$ and item-feature matrix $Q$ with small random values.

In [3]:
def initialize_matrices(n_users, n_items, n_factors):
    """
    Initializes the user-feature (P) and item-feature (Q) matrices.

    Args:
        n_users (int): Number of users.
        n_items (int): Number of items.
        n_factors (int): Number of latent factors.

    Returns:
        tuple: A tuple containing:
            - P (np.ndarray): The user-feature matrix (n_users x n_factors).
            - Q (np.ndarray): The item-feature matrix (n_items x n_factors).
    """
    # TODO: Initialize P and Q with small random values from a standard normal distribution.
    P = np.random.normal(scale=0.1, size=(n_users, n_factors))
    Q = np.random.normal(scale=0.1, size=(n_items, n_factors))
    return (P,Q)


### 4.2 The Training Loop (SGD)

This is the core of our algorithm. We will loop for a specified number of epochs. In each epoch, we will iterate over all known ratings in our training set `R_train` and update the corresponding user and item vectors in `P` and `Q`.

In [4]:
def train_model(R_train, P, Q, learning_rate, regularization, epochs):
    R_train = np.nan_to_num(R_train, nan=0.0)
    r_predict = np.zeros(R_train.shape)
    e = np.zeros(R_train.shape)
    n_users, n_items = R_train.shape
    for epoch in range(epochs):
        L=0
        for u in range(n_users):
            for i in range(n_items):
                if R_train[u][i] > 0:
                    r_predict[u][i] = P[u].dot(Q[i])
                    e[u][i]= R_train[u][i]-r_predict[u][i]
                    p_u_old = P[u].copy()
                    P[u]=P[u]+learning_rate*(e[u][i]*Q[i]-regularization*P[u])
                    Q[i]=Q[i]+learning_rate*(e[u][i]*p_u_old-regularization*Q[i])
                    L=L+(e[u][i]*e[u][i])+regularization*(np.sum(P[u]**2)+np.sum(Q[i]**2))
        print ("Epoch",epoch,"loss",L)
    
                
    return (P,Q)
                
                

            
        
    """
    Trains the matrix factorization model using SGD.

    Args:
        R_train (np.ndarray): The training user-item matrix.
        P (np.ndarray): The user-feature matrix.
        Q (np.ndarray): The item-feature matrix.
        learning_rate (float): The learning rate (alpha).
        regularization (float): The regularization parameter (lambda).
        epochs (int): The number of iterations over the training data.

    Returns:
        tuple: A tuple containing the trained P and Q matrices.
    """

---
## Part 5: Evaluation

After training, we must evaluate how well our model performs on unseen data. We will use the **Root Mean Squared Error (RMSE)**, which measures the average magnitude of the errors between predicted and actual ratings.

The formula is:
$$RMSE = \sqrt{\frac{1}{|\mathcal{T}|} \sum_{(u,i) \in \mathcal{T}} (r_{ui} - \hat{r}_{ui})^2}$$

Where $\mathcal{T}$ is the set of ratings in our test set. A lower RMSE means better performance.

In [5]:
def calculate_rmse(R_test, P, Q):
    R_test = np.nan_to_num(R_test, nan=0.0)
    r_predict = np.zeros(R_test.shape)
    error=0
    T=0
    n_users, n_items = R_test.shape
    for u in range(n_users):
        for i in range(n_items):
            if R_test[u][i] > 0:
                r_predict[u][i] = P[u].dot(Q[i])
                error= error+ (R_test[u][i]-r_predict[u][i])**2
                T=T+1
    RMSE=np.sqrt(error/T)
    return (RMSE)
                
    """
    Calculates the Root Mean Squared Error (RMSE) on the test set.

    Args:
        R_test (np.ndarray): The testing user-item matrix.
        P (np.ndarray): The trained user-feature matrix.
        Q (np.ndarray): The trained item-feature matrix.

    Returns:
        float: The RMSE value.
    """

---
## Part 6: Putting It All Together

Now, let's connect all the pieces. We'll set our hyperparameters, initialize our matrices, train the model, and finally evaluate it.

**Your Goal:** Tune the hyperparameters to achieve an **RMSE below 0.98**. A good model can even reach ~0.95. If your RMSE is higher, try adjusting the learning rate, regularization, number of factors, or epochs.

In [6]:
# --- Hyperparameters ---
# Number of latent factors (k)
# Learning rate (alpha)
# Regularization parameter (lambda)
# Number of epochs
k=50
learning_rate=0.01
regularization=0.04
epochs=50


# --- Initialization ---
# Remember user and item IDs are 1-based, but our numpy arrays are 0-based.
# The number of users/items from the shape of R_df is correct for 0-based indexing.

R_train,R_test=create_user_item_matrix(df)
n_users,n_items= R_train.shape
P,Q= initialize_matrices(n_users, n_items, k)



# --- Training ---
P_train,Q_train=train_model(R_train, P, Q, learning_rate, regularization, epochs)


# --- Evaluation ---
RMSE=calculate_rmse(R_test, P_train,Q_train)
print(RMSE)


Epoch 0 loss 687292.6986978222
Epoch 1 loss 186748.80424357997
Epoch 2 loss 112689.6091035388
Epoch 3 loss 100883.27571388651
Epoch 4 loss 95945.00862286352
Epoch 5 loss 92722.09910743852
Epoch 6 loss 90139.86972731406
Epoch 7 loss 87865.60185511323
Epoch 8 loss 85785.93984287042
Epoch 9 loss 83848.94630109498
Epoch 10 loss 82021.13550095049
Epoch 11 loss 80282.80109909673
Epoch 12 loss 78626.74972327506
Epoch 13 loss 77053.80921299422
Epoch 14 loss 75568.03979182869
Epoch 15 loss 74173.5156795197
Epoch 16 loss 72872.72321723425
Epoch 17 loss 71666.05000244774
Epoch 18 loss 70551.87355242026
Epoch 19 loss 69526.91977132851
Epoch 20 loss 68586.70088252633
Epoch 21 loss 67725.93518443446
Epoch 22 loss 66938.90593298404
Epoch 23 loss 66219.74667994933
Epoch 24 loss 65562.65530388737
Epoch 25 loss 64962.04536863662
Epoch 26 loss 64412.64550202787
Epoch 27 loss 63909.55753453382
Epoch 28 loss 63448.28328773344
Epoch 29 loss 63024.728633461935
Epoch 30 loss 62635.191987418104
Epoch 31 loss 6

---

## Part 7: Making Recommendations

The ultimate goal is to recommend movies! Now that we have our trained matrices $P$ and $Q$, we can predict the rating for *any* user-item pair, including those the user has not seen yet.

The process for a given user `user_id`:
1.  Get the user's latent vector $p_u$ from the trained matrix $P$.
2.  Calculate the predicted ratings for all items by taking the dot product of $p_u$ and the entire item-feature matrix $Q^T$.
3.  Create a list of movie titles and their predicted ratings.
4.  Filter out movies the user has already seen.
5.  Sort the remaining movies by their predicted rating in descending order.
6.  Return the top N movies.

In [7]:
def recommend_top_movies(user_id, P, Q, movie_titles_df, R_df, top_n=10):
    """
    Recommends top N movies for a given user.

    Args:
        user_id (int): The ID of the user.
        P (np.ndarray): The trained user-feature matrix.
        Q (np.ndarray): The trained item-feature matrix.
        movie_titles_df (pd.DataFrame): Dataframe with item_id and title.
        R_df (pd.DataFrame): The original user-item matrix dataframe (for checking seen movies).
        top_n (int): The number of movies to recommend.

    Returns:
        pd.DataFrame: A dataframe with the top N recommended movie titles and their predicted ratings.
    """
    movie_titles = movie_titles_df['title'].values
    R = R_df.values

    num_items = R.shape[1]
    r_predict = np.full(num_items, -np.inf) 

    for i in range(num_items):
        if np.isnan(R[user_id, i]):
            r_predict[i] = P[user_id].dot(Q[i])

    top_indices = np.argsort(r_predict)[::-1][:top_n]

    recommendations = np.vstack((movie_titles[top_indices],  np.round(r_predict[top_indices], 2))).T

    return recommendations
    
    

In [8]:
def create_R_df(df):
    df=df[["user_id","rating","title"]]
    R_df = df.pivot_table(index="user_id", columns="title", values="rating")
    
    return (R_df)

R_df=create_R_df(df)

In [9]:
recommendations= recommend_top_movies(1, P_train, Q_train, df, R_df)

In [10]:
print (recommendations)

[['Rock, The (1996)' 4.85]
 ['Adventures of Priscilla, Queen of the Desert, The (1994)' 4.84]
 ['Last of the Mohicans, The (1992)' 4.81]
 ['Cliffhanger (1993)' 4.81]
 ['Rock, The (1996)' 4.75]
 ["Marvin's Room (1996)" 4.73]
 ['Mirror Has Two Faces, The (1996)' 4.71]
 ['Taxi Driver (1976)' 4.7]
 ['Jaws (1975)' 4.68]
 ['Terminator, The (1984)' 4.68]]
